In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import emd
from scipy.signal import hilbert
from scipy import ndimage

In [2]:
output_dir = "./00007656_s010_t000_processed_data"
results_dir = "./results"
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [3]:
fs = 256  # Frequência de amostragem
channel = "EEG_F3-REF"  # Canal a ser analisado

# HHSA para um paciente e um canal

In [4]:
def holospectrum_single_channel(data, fs=256):
    """
    Aplica Holo-Hilbert Spectral Analysis (HHSA) ao sinal de EEG de um único canal.
    """
    # Garantir que o dado seja uma matriz 2D com dimensões corretas
    if data.ndim == 1:
        data = data.reshape(-1, 1)  # Reshape para (n_samples, 1)
    
    # Configuração da Decomposição
    config = emd.sift.get_config('mask_sift')
    config['max_imfs'] = 4
    config['mask_freqs'] = 'if'
    config['mask_amp_mode'] = 'ratio_imf'
    config['imf_opts/sd_thresh'] = 0.5
    config['mask_step_factor'] = 5

    # Decomposição usando Mask SIFT
    imf = emd.sift.mask_sift(data, **config)

    # Transformada de Hilbert
    IP, IF, IA = emd.spectra.frequency_transform(imf, fs, 'nht')

    # Segunda camada de decomposição
    masks = np.array([25 / 2**ii for ii in range(12)]) / fs
    imf2 = emd.sift.mask_sift_second_layer(IA, masks, sift_args=config)

    # Transformada de Hilbert na segunda camada
    IP2, IF2, IA2 = emd.spectra.frequency_transform(imf2, fs, 'nht')

    # Configuração do histograma
    carrier_hist = (1, 100, 128, 'log')
    am_hist = (1e-2, 100, 128, 'log')

    # Computar HHT
    fcarrier, hht = emd.spectra.hilberthuang(IF, IA, carrier_hist, sum_time=False)
    shht = ndimage.gaussian_filter(hht, 2)

    # Garantir que IF, IF2 e IA2 tenham as dimensões corretas
    IF = np.broadcast_to(IF, IF2.shape)
    IF2 = IF2.squeeze()
    IA2 = IA2.squeeze()

    # Computar HHSA
    fcarrier, fam, holo = emd.spectra.holospectrum(IF, IF2, IA2, carrier_hist, am_hist)
    sholo = ndimage.gaussian_filter(holo, 1)

    np.nan_to_num(sholo, copy=False)

    # Computar o espectro de potência
    spec = np.mean(IA**2, axis=0)

    return fcarrier, fam, sholo, shht, spec

In [5]:
def process_single_channel(output_dir, channel, fs=256, result_dir='./results'):
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)

    # Carregar o sinal
    signal_path = os.path.join(output_dir, f"{channel}.npy")
    signal = np.load(signal_path)

    # Garantir que o sinal seja 1D
    if signal.ndim != 1:
        signal = signal.flatten()

    fcarrier, fam, sholo, shht, spec = holospectrum_single_channel(signal, fs)
    
    # Plotando os resultados
    fig, axes = plt.subplots(2, 2, figsize=(20, 20))

    # HHT
    cp = axes[0, 0].contourf(fcarrier, fcarrier, shht.T, cmap='jet', levels=40)
    axes[0, 0].set_yscale('log')
    axes[0, 0].set_xscale('log')
    axes[0, 0].set_title('Hilbert-Huang Transform')
    axes[0, 0].set_xlabel('Frequency (Hz)')
    axes[0, 0].set_ylabel('Frequency (Hz)')
    plt.colorbar(cp, ax=axes[0, 0])

    # HHSA
    cp = axes[0, 1].contourf(fcarrier, fam, sholo.T, cmap='jet', levels=40)
    axes[0, 1].set_yscale('log')
    axes[0, 1].set_xscale('log')
    axes[0, 1].set_title('Holo-Hilbert Spectrum')
    axes[0, 1].set_xlabel('FM Frequency (Hz)')
    axes[0, 1].set_ylabel('AM Frequency (Hz)')
    plt.colorbar(cp, ax=axes[0, 1])

    # Marginal Spectrum
    axes[1, 0].plot(fcarrier, np.sum(shht, axis=1))
    axes[1, 0].set_xscale('log')
    axes[1, 0].set_title('Marginal Spectrum')
    axes[1, 0].set_xlabel('Frequency (Hz)')
    axes[1, 0].set_ylabel('Amplitude')

    # Power Spectrum
    axes[1, 1].plot(fcarrier, spec)
    axes[1, 1].set_xscale('log')
    axes[1, 1].set_yscale('log')
    axes[1, 1].set_title('Power Spectrum')
    axes[1, 1].set_xlabel('Frequency (Hz)')
    axes[1, 1].set_ylabel('Power')

    plt.tight_layout()
    plt.savefig(f'{result_dir}/holo_hilbert_{channel}.png', dpi=100, bbox_inches='tight')
    plt.close()

    print(f"Holo-Hilbert Spectral Analysis concluído para o canal {channel}!")


In [6]:
# Aplicação no dado
process_single_channel(output_dir, channel,fs)

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (258048,4)  and requested shape (258048,4,4)